## An example workflow for schism

This is a simple example for SCHISM that assumes that the grid file is given. Since no input options for meteo or bathymetry is given they are retrieved from an erddap server (GFS, srtmplus).

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon

In [ ]:
cpath = pyPoseidon.__path__[0].split('/lib/')[0] # get the current kernel path

In [ ]:
os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

In [ ]:
import pyPoseidon.model as pmodel
from pyPoseidon.utils import data,point,obs
import os
import glob
import xarray as xr
import geopandas as gp
import cartopy.feature as cf

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
from hvplot import xarray, pandas

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
#optional
#plt.style.use("dark_background")

## Choose data path

In [ ]:
# Set the env PATH for Schism executable. If you don't the conda path is assumed.
#os.environ['SCHISM'] =  /path_to_schism_executable

In [ ]:
DATA_PATH = os.path.dirname(pyPoseidon.__file__)   # get path to the test data installed with the package
DATA_PATH

..or define a local path

In [ ]:
DATA_PATH = os.getcwd() 
DATA_PATH  = os.path.abspath(os.path.join(DATA_PATH, os.pardir)) + '/pyPoseidon/'
DATA_PATH

In [ ]:
PWD = os.getcwd()
PWD

### Define case

This is a simple case with 12h time frame. It is the first step for the Hindcast_Schism.ipynb

In [ ]:
#define in a dictionary the properties of the model..
case={'solver':'schism',
     'geometry':{'lon_min' : -25,'lon_max' : -13.,'lat_min' : 61.,'lat_max' : 69.}, 
     'manning':.12,
     'windrot':0.00001,
     'tag':'schism',
     'start_date':'2017-10-1 0:0:0',
     'time_frame':'12H',
     'meteo_source' : [DATA_PATH + '/tests/data/erai.grib'], #meteo files
     'meteo_engine':'cfgrib',
     'dem_source' : DATA_PATH + '/tests/data/dem.nc', #path for dem file
     'ncores': 4 , #number of cores
     'rpath':PWD + '/test/schism/', #location of calc folder
     'update':['all'], #set which component should be updated  (meteo,dem,model)
     'monitor':True, # get time series for observation points
     'parameters':{'dt':400, 'rnday':.5, 'hotout':1, 'ihot':0,'nspool':9, 'ihfskip':36, 'hotout_write':108 } # set param.in components
    }

## Initialize

In [ ]:
#initialize a model
b = pmodel(**case)

## Step by Step process

In [ ]:
b.create()

In [ ]:
b.output()

In [ ]:
b.save()

In [ ]:
b.set_obs()

In [ ]:
b.run()

## ... or simply in one step


In [ ]:
#initialize a model
b = pmodel(**case)
b.execute()

## Results

In [ ]:
b.get_data() # combine output

In [ ]:
out = b.data.Dataset

In [ ]:
out.dahv

In [ ]:
out.depth.compute().max(), out.depth.compute().min()

In [ ]:
out.pplot.contourf(var='depth' ,vmin=out.depth.min(), vmax=out.depth.max())

In [ ]:
out.pplot.quiver(var='dahv',it=4,scale=.01)

### with hvplot

In [ ]:
out.hplot.contourf(var='depth' , vmin=out.depth.min(), vmax=out.depth.max())

In [ ]:
out.hplot.grid(lw=.3, markersize=1)

### Animate

In [ ]:
v = out.pplot.frames(var='elev',title='SSH')#, coastlines=b.coastlines)#,vmin=-.5, vmax=.5, nv=20)
v

In [ ]:
v.save('test/elev.mp4')

In [ ]:
out.pplot.qframes(var='wind_speed',title='Vel',scale=30.)